In [1]:
import sys
from typing import *
from time import sleep, time
from random import seed
from io import StringIO
from sys import settrace
from sympy import isprime
from random import randint
from math import log, floor
from itertools import chain
from functools import reduce
from socket import gethostname
from joblib import Parallel, delayed
from multiprocessing import cpu_count
from contextlib import redirect_stdout
from timeit import default_timer as timer
from multiprocessing.pool import ThreadPool
sys.version

'3.9.7 (default, Sep 16 2021, 08:50:36) \n[Clang 10.0.0 ]'

In [2]:
def generate_primes(domain: int=1000*1000, num_attempts: int=1000) -> list[int]:
    primes: set[int] = set()
    seed(time())
    for _ in range(num_attempts):
        candidate: int = randint(4, domain)
        if isprime(candidate):
            primes.add(candidate)
    return sorted(primes)

print(len(generate_primes()))

77


In [3]:
# this version used thread parallelim,
# which doesn't work very well because of Python
# Global Interpreter Lock (GIL)
# 
# def parallel(func=None, args=(), merge_func=lambda x:x, parallelism = cpu_count()):
#     def decorator(func: Callable):
#         def inner(*args, **kwargs):
#             pool = ThreadPool(parallelism)
#             results = [pool.apply_async(func, args, kwargs) for _ in range(parallelism)]
#             results = [r.get() for r in results]
#             pool.close()
#             pool.join()
#             return merge_func(results)
#         return inner
#     if func is None:
#         # decorator was used like @parallel(...)
#         return decorator
#     else:
#         # decorator was used like @parallel, without parens
#         return decorator(func)

# this version uses process parallelism,
# so the GIL is not a problem, but it's more
# expensive, it only makes sense for bigger jobs
#
def parallel(func=None, args=(), merge_func=lambda x:x, parallelism = cpu_count()):
    def decorator(func: Callable):
        def inner(*args, **kwargs):
            results = Parallel(n_jobs=parallelism)(delayed(func)(*args, **kwargs) for i in range(parallelism))
            return merge_func(results)
        return inner
    if func is None:
        # decorator was used like @parallel(...)
        return decorator
    else:
        # decorator was used like @parallel, without parens
        return decorator(func)

@parallel(merge_func=lambda li: sorted(set(chain(*li))))
def generate_primes(domain: int=1000*1000, num_attempts: int=1000) -> list[int]:
    primes: set[int] = set()
    seed(time())
    for _ in range(num_attempts):
        candidate: int = randint(4, domain)
        if isprime(candidate):
            primes.add(candidate)
    return sorted(primes)

print(len(generate_primes()))

1233


In [4]:
def foo1():
    print('Running in production, touching databases!')

foo1()

Running in production, touching databases!


In [5]:
# production_servers = []
production_servers = ['Spacegray.lan'] # this is my hostname, for testing

def production(func: Callable):
    def inner(*args, **kwargs):
        if gethostname() in production_servers:
            return func(*args, **kwargs)
        else:
            print('This host is not a production server, skipping function decorated with @production...')
    return inner

def development(func: Callable):
    def inner(*args, **kwargs):
        if gethostname() not in production_servers:
            return func(*args, **kwargs)
        else:
            print('This host is a production server, skipping function decorated with @development...')
    return inner

def inactive(func: Callable):
    def inner(*args, **kwargs):
        print('Skipping function decorated with @inactive...')
    return inner

@production
def foo2():
    print('Running in production, touching databases!')

foo2()

@development
def foo3():
    print('Running in production, touching databases!')
    
foo3()

@inactive
def foo4():
    print('Running in production, touching databases!')
    
foo4()

Running in production, touching databases!
This host is a production server, skipping function decorated with @development...
Skipping function decorated with @inactive...


In [6]:
def deployable(func):
    def inner(*args, **kwargs):
        if 'deploy' in kwargs:
            if kwargs['deploy'].lower() in ['production', 'prod'] and gethostname() not in production_servers:
                print('This host is not a production server, skipping...')
                return
            if kwargs['deploy'].lower() in ['development', 'dev'] and gethostname() not in development_servers:
                print('This host is not a development server, skipping...')
                return
            if kwargs['deploy'].lower() in ['skip', 'none']:
                print('Skipping...')
                return
            del kwargs['deploy'] # to avoid func() throwing an unexpected keyword exception
        return func(*args, **kwargs)
    return inner

@deployable
def foo_generic():
    print('Running in production, touching databases!')

foo_generic(deploy='prod')

Running in production, touching databases!


In [7]:
def print_lines1(num_lines):
    for i in range(num_lines):
        print(f'Line #{i+1}')

print_lines1(3)
print()

def redirect(func=None, line_print: Callable = None):
    def decorator(func: Callable):
        def inner(*args, **kwargs):
            with StringIO() as buf, redirect_stdout(buf):
                func(*args, **kwargs)
                output = buf.getvalue()
            lines = output.splitlines()
            if line_print is not None:
                for line in lines:
                    line_print(line)
            else:
                width = floor(log(len(lines), 10)) + 1
                for i, line in enumerate(lines):
                    i += 1
                    print(f'{i:0{width}}: {line}')
        return inner
    if func is None:
        # decorator was used like @redirect(...)
        return decorator
    else:
        # decorator was used like @redirect, without parens
        return decorator(func)

@redirect
def print_lines2(num_lines):
    for i in range(num_lines):
        print(f'Line #{i+1}')

print_lines2(10)
print()

lines = []
def save_lines(line):
    lines.append(line)
    
@redirect(line_print=save_lines)
def print_lines3(num_lines):
    for i in range(num_lines):
        print(f'Line #{i+1}')

print_lines3(3)
print(lines)

Line #1
Line #2
Line #3

01: Line #1
02: Line #2
03: Line #3
04: Line #4
05: Line #5
06: Line #6
07: Line #7
08: Line #8
09: Line #9
10: Line #10

['Line #1', 'Line #2', 'Line #3']


In [8]:
def stacktrace(func=None, exclude_files=['anaconda']):
    def tracer_func(frame, event, arg):
        co = frame.f_code
        func_name = co.co_name
        caller_filename = frame.f_back.f_code.co_filename
        if func_name == 'write':
            return # ignore write() calls from print statements
        for file in exclude_files:
            if file in caller_filename:
                return # ignore in ipython notebooks
        args = str(tuple([frame.f_locals[arg] for arg in frame.f_code.co_varnames]))
        if args.endswith(',)'):
            args = args[:-2] + ')'
        if event == 'call':
            print(f'--> Executing: {func_name}{args}')
            return tracer_func
        elif event == 'return':
            print(f'--> Returning: {func_name}{args} -> {repr(arg)}')
        return
    def decorator(func: Callable):
        def inner(*args, **kwargs):
            settrace(tracer_func)
            func(*args, **kwargs)
            settrace(None)
        return inner
    if func is None:
        # decorator was used like @stacktrace(...)
        return decorator
    else:
        # decorator was used like @stacktrace, without parens
        return decorator(func)

def b():
    print('...')

@stacktrace
def a(arg):
    print(arg)
    b()
    return 'world'

a('foo')

--> Executing: a('foo')
foo
--> Executing: b()
...
--> Returning: b() -> None
--> Returning: a('foo') -> 'world'


In [9]:
def traceclass(cls: type):
    def make_traced(cls: type, method_name: str, method: Callable):
        def traced_method(*args, **kwargs):
            print(f'--> Executing: {cls.__name__}::{method_name}()')
            return method(*args, **kwargs)
        return traced_method
    for name in cls.__dict__.keys():
        if callable(getattr(cls, name)) and name != '__class__':
            setattr(cls, name, make_traced(cls, name, getattr(cls, name)))
    return cls

@traceclass
class Foo:
    i: int = 0
    def __init__(self, i: int = 0):
        self.i = i
    def increment(self):
        self.i += 1
    def __str__(self):
        return f'This is a {self.__class__.__name__} object with i = {self.i}'

f1 = Foo()
f2 = Foo(4)
f1.increment()
print(f1)
print(f2)

--> Executing: Foo::__init__()
--> Executing: Foo::__init__()
--> Executing: Foo::increment()
--> Executing: Foo::__str__()
This is a Foo object with i = 1
--> Executing: Foo::__str__()
This is a Foo object with i = 4


In [10]:
def measure(func: Callable):
    def inner(*args, **kwargs):
        start = timer()
        func(*args, **kwargs)
        elapsed_sec = timer() - start
        print(f'Elapsed: {elapsed_sec:.3f} secs')
    return inner

@measure
def sleeper(seconds: int = 0):
    print('Going to sleep...')
    sleep(seconds)
    print('Done!')

sleeper(3)

Going to sleep...
Done!
Elapsed: 3.003 secs
